# Authentication: Authenticate and obtain access tokens

In [ ]:
import globus_sdk
import json
from globus_sdk.scopes import TransferScopes

# Enter the CLIENT ID generated when registering your Native App with Globus
CLIENT_ID = ""

# AUTHENTICATION
auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
auth_client.oauth2_start_flow(requested_scopes=TransferScopes.all)
authorize_url = auth_client.oauth2_get_authorize_url()
print(f"Please go to this URL and login:\n\n{authorize_url}\n")

#Enter authorization code
auth_code = input("Please enter the code here: ").strip()
tokens = auth_client.oauth2_exchange_code_for_tokens(auth_code)

#Token Handling: Get tokens
transfer_tokens = tokens.by_resource_server["transfer.api.globus.org"]

# Token Handling: Save the refresh token and access token for later use
saved_tokens = {
    "refresh_token": transfer_tokens["refresh_token"],
    "access_token": transfer_tokens["access_token"],
    "expires_at_seconds": transfer_tokens["expires_at_seconds"]
}

# Token Handling: Function to update the saved tokens when they are refreshed
def update_tokens(token_response):
    saved_tokens["access_token"] = token_response.by_resource_server['transfer.api.globus.org']['access_token']
    saved_tokens["expires_at_seconds"] = token_response.by_resource_server['transfer.api.globus.org']['expires_at_seconds']

# Token Handling: Use RefreshTokenAuthorizer to automatically handle token refresh
authorizer = globus_sdk.RefreshTokenAuthorizer(
    saved_tokens["refresh_token"],
    auth_client,
    access_token=saved_tokens["access_token"],
    expires_at=saved_tokens["expires_at_seconds"],
    on_refresh=update_tokens,
)


# Use the SDK functions to create a transfer task that copies file from Quest to my local machine that is running Globus Connect Personal

In [ ]:
# Construct the TransferClient using the Tokens provided by authorizer
transfer_client = globus_sdk.TransferClient(authorizer=authorizer)

# Enter the source collection UUID for your transfer
source_endpoint_id = ""

# Enter the destination collection UUID for your transfer
dest_endpoint_id = ""

# create a Transfer task consisting of one or more items
task_data = globus_sdk.TransferData(
    source_endpoint=source_endpoint_id, destination_endpoint=dest_endpoint_id
)

task_data.add_item(
    "/",  # source file path
    "",  # destination file path
)


# Submit the transfer task for execution

In [ ]:
# submit, getting back the task ID
task_doc = transfer_client.submit_transfer(task_data)
task_id = task_doc["task_id"]
print(f"submitted transfer, task_id={task_id}")

# Check the transfer task status (high level)

In [ ]:
# Get and print the task's status
task = transfer_client.get_task(task_id)
print(f"Task Status: {task['status']}")

# Check the transfer task status (detailed)

In [ ]:
def get_transfer_results(transfer_client, task_id):
    # Get detailed task information
    task = transfer_client.get_task(task_id).data
    print("Task Information:")
    print(json.dumps(task, indent=2))

    # Get list of events for the task
    events_response = transfer_client.task_event_list(task_id)
    events = events_response['DATA']
    print("\nTask Events:")
    for event in events:
        print(json.dumps(event, indent=2))

    # Get list of successful files
    successful_files_response = transfer_client.task_successful_transfers(task_id)
    successful_files = successful_files_response['DATA']
    print("\nSuccessful Files:")
    for file in successful_files:
        print(json.dumps(file, indent=2))

    # Get list of skipped errors
    skipped_errors_response = transfer_client.task_skipped_errors(task_id)
    skipped_errors = skipped_errors_response['DATA']
    print("\nSkipped Errors:")
    for error in skipped_errors:
        print(json.dumps(error, indent=2))

get_transfer_results(transfer_client, task_id)